In [1]:
import tpx
import numpy

from tools.visualization import slice_and_pack_data

In [2]:
# Select a file to load:
fname = "/home/cadams/CRAB/UTA-tpx3/120thresh_12biasvolt_IIgain0V000000.tpx3"

In [3]:
#Load the file from the binary data:
crab0_data = tpx.frame(fname=fname)

In [4]:
crab0_data.shape
print(crab0_data["TOA"])

[0.06795924 0.06795927 0.07022654 ... 0.04742079 0.0474206  0.04742034]


In [5]:
# Apply a threshold to remove events with too little charge:
crab0_data = crab0_data.threshold(26)
print(crab0_data.shape)

(169258,)


In [6]:
# What is the min and max time?
min_time = crab0_data['TOA'].min()
max_time = crab0_data['TOA'].max()
print(min_time)
print(max_time)

0.0
26.842255728125


 alpha events are coming at something like 137HZ
... so to resolve individual ones we need to stretch this out.

1/137 (huh ... alpha) is about 0.007 s between alphas, on average, so we want that to be broadly discriminated.
So, aim for 

In [7]:
time_step = 0.001 # HOw much time, in s, rendered at once






In [8]:
# Load the vizualization modules:

from vispy import app, scene, io
from vispy.scene import visuals
from vispy.color import get_colormaps, BaseColormap
from vispy.visuals import TextVisual, LineVisual

%gui qt

In [9]:
# Create a color map to render the data more nicely:
from matplotlib import cm

In [10]:
cmap = cm.get_cmap("gist_stern_r")

/tmp/ipykernel_36527/3150136637.py:1: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap("gist_stern_r")


In [11]:
# Create a 3D window for vizualing the data:

# Rendering steps:
canvas = scene.SceneCanvas(keys='interactive', size=(1600, 1600), show=True)
canvas.measure_fps()
view = canvas.central_widget.add_view()
view.camera = 'turntable'
view.padding=100

# # Add the detector box:
# lines_top = numpy.zeros((4,3))
# lines_top[0][0] = -256; lines_top[0][1] = -256;
# lines_top[1][0] = -256; lines_top[1][1] = 256;
# lines_top[2][0] = 256; lines_top[2][1] = 256;
# lines_top[3][0] = 256; lines_top[3][1] = -256;

# axis = LineVisual(pos=lines_top)
# view.add(axis)

cube = scene.visuals.Box(1,0,1,parent=view.scene)

scatter = visuals.Markers(symbol='square', scaling="scene")
view.add(scatter)

i = 1

i = 0
time_step = 0.001 # HOw much time, in s, rendered at once
total_window = 0.02 # in seconds
drift_scale = 2e1

# slice_data = crab0_data.time_slice(0.25,0.5)

initial_xyz, initial_z = slice_and_pack_data(crab0_data, 
                                             min_time = 0, 
                                             max_time = total_window, 
                                             drift_scale=drift_scale, 
                                             shift_to_min=True)
print(initial_xyz.shape)
print(numpy.min(initial_xyz[:,0]))
print(numpy.min(initial_xyz[:,1]))

print(initial_z)

colors = cmap(initial_z / 1000.)


scatter.set_data(initial_xyz, edge_width=0, face_color=colors, size=1./256, symbol="square")


def update(ev, drift_scale=drift_scale):
    global i


    if (i)*time_step > numpy.max(crab0_data["TOA"]): 
        i = 0



    xyz, z = slice_and_pack_data(crab0_data, 
                                 min_time = i*time_step, 
                                 max_time = (i+1)*time_step + total_window, 
                                 drift_scale = drift_scale,
                                 )
    colors = cmap(z / 1000.)
    
    i += 1

    scatter.set_data(xyz, edge_width=0, face_color=colors, size=1./256, symbol="square")
        
timer = app.Timer()
timer.connect(update)
# slow this down a bit to better see what happens
timer.start()


(114, 3)
-0.11328125
0.03515625
[175. 175.  75. 150. 300. 300. 125. 150. 275.  50.  75. 175. 125. 200.
  50.  75. 150. 200. 200. 100. 125. 150. 325. 150. 250. 275. 100. 200.
 250.  75. 125. 250. 325.  50. 125.  50. 150.  75.  75.  50.  50. 325.
  75. 150. 125. 250. 100. 150. 100. 175. 275. 100. 125. 125.  50. 200.
  75. 300.  75. 375. 225. 200. 225. 300. 350.  75.  75. 125.  75. 100.
 200. 175.  50. 125.  50. 150. 175. 425. 200. 275.  75. 175. 250.  75.
 225.  75. 200.  75. 200. 275.  75. 250. 200.  50. 175. 100. 225. 225.
 200. 350.  75. 200. 150. 200.  50. 350.  50. 225. 500. 325. 675. 200.
 125.  75.]


53.6 FPS
60.0 FPS
59.9 FPS
60.0 FPS
